# MNIST 入门

## The MNIST Data


In [5]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


MINIST 数据被分为5500 train，10000test， 5000 validation；
数据点包括：一张image，一个label

每张图是 28 * 28 像素，可以理解为一个矩阵
![](https://www.tensorflow.org/images/MNIST-Matrix.png)

flatten the data into a vector 28*28=784;

`mnist.train.images`是一个tensor，shape为[55000, 784] ；
![](https://www.tensorflow.org/images/mnist-train-xs.png)

使用one-hot-encoding；0到9十个数字，因此,mnist.train.labels是一个[55000, 10]的array。
![](https://www.tensorflow.org/images/mnist-train-ys.png)

## softmax regression
softmax来做分类；
$$evidence_i = \sum_j W_{i,j}x_j + b_i$$
$$y=softmax(evidence)$$

用softmax作为最后输出层的激活函数，即
$$p_i = softmax(x)_i = \frac{exp(WX+b)}{\sum_j exp(WX+b)}$$;

是logistic function的推广，将输出转化为各个label上的概率值；文档中的图片给出了其原理：
![](https://www.tensorflow.org/images/softmax-regression-scalargraph.png)
![](https://www.tensorflow.org/images/softmax-regression-scalarequation.png)
![](https://www.tensorflow.org/images/softmax-regression-vectorequation.png)

## 具体实现
矩阵计算十分耗时，tensorflow采取先将所有运算都事先生成好的方式，最后一步地用底层C++实现其运算。

In [1]:
import tensorflow as tf

以下使用placeholder，标志一个占位符，在后面的回话中可以feed其一个值。此处想要input任何一个MNIST image; shape指定为[None, 784].(此处None表示任意长度)

In [2]:
x = tf.placeholder(tf.float32, shape= [None, 784])

适用Variable来储存权重与偏置，因为他们需要被更新。

In [3]:
w = tf.Variable(tf.zeros([784, 10]), name = 'weights')
b = tf.Variable(tf.zeros([10]), name = 'biases')

y = tf.nn.softmax(tf.matmul(x, w) + b) # 就这样定义好了输出

### train
机器学习三要素：模型 策略 算法；
模型是softmax 回归，策略是使用哪个loss function呢？算法是使用哪种迭代方式呢？

交互熵 (cross-entropy): $h_{y^{'}}(y) = -\sum_i y_i^{'} log(y_i)$

其中$y^{,}$是实际label，$y$是我们计算出来的概率分布

在model中要告诉它我们要使用交互熵作为loss function了

In [6]:
# y_表示实际的label
y_ = tf.placeholder(tf.float32, [None, 10])

# 定义一个op作为loss 
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# 需要指定optimizer了, 学习率为0.5的梯度下降法
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.InteractiveSession()

# 初始化所有Variable
tf.global_variables_initializer().run()

# 训练1000次
for _ in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100) # 每次拿出100个
    sess.run(train_step, feed_dict = {x:batch_xs, y_:batch_ys})

使用少量的随机抽取数据进入batch，是随机梯度下降。如果每一步都使用所有的数据，计算比较慢。

### 评价我们的模型
softmax函数输出的是分到10个label的概率值，即预测的分类应该是$argmax Y$


In [12]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9134
